In [6]:
import numpy as np
import numpy.linalg as npla

In [7]:
def nf2DualLP(filename):

    # assumes that first row is source and last is sink like in the question
    # edges will be numbered as if they are being read row-by-row left to right
    # vertices will be numbered by row

    nf = np.loadtxt(filename)
    for i in range(nf.shape[0]):
        nf[i, i] = 0
    numedges = np.count_nonzero(nf)
    numvertices = nf.shape[0] - 2     # non terminal vertices
    numslacks = numedges
    slack_counter = 0
    edge_counter = 0
    dual_constraints = np.zeros(
        (numedges, numedges + numvertices + numslacks + 1))
    obj = np.zeros(2 * numedges + numvertices)

    for i in range(numvertices + 2):
        for j in range(numvertices + 2):
            if nf[i, j] != 0:

                obj[edge_counter] = nf[i, j]

                if i == 0:
                    dual_constraints[edge_counter, edge_counter] = 1
                    dual_constraints[edge_counter, numedges + j - 1] = 1
                    dual_constraints[edge_counter, numedges +
                                     numvertices + slack_counter] = -1
                    dual_constraints[edge_counter, -1] = 1
                    edge_counter += 1
                    slack_counter += 1

                elif j == numvertices + 1:
                    dual_constraints[edge_counter, edge_counter] = 1
                    dual_constraints[edge_counter, numedges + i - 1] = -1
                    dual_constraints[edge_counter, numedges +
                                     numvertices + slack_counter] = -1
                    dual_constraints[edge_counter, -1] = 0
                    edge_counter += 1
                    slack_counter += 1

                else:
                    dual_constraints[edge_counter, edge_counter] = 1
                    dual_constraints[edge_counter, numedges + i - 1] = -1
                    dual_constraints[edge_counter, numedges + j - 1] = 1
                    dual_constraints[edge_counter, numedges +
                                     numvertices + slack_counter] = -1
                    edge_counter += 1
                    slack_counter += 1

    sign_constraints = np.block([
        [np.eye(numedges), np.zeros((numedges, numvertices + numslacks + 1))],
        [np.zeros((numslacks, numedges + numvertices)),
         np.eye(numslacks), np.ones(numedges).reshape(1, numedges).T]
    ])

    LPMatrix = np.vstack((dual_constraints, sign_constraints))

    return LPMatrix, obj


def nf2PrimalLP(filename):
    nf = np.loadtxt(filename)
    for i in range(nf.shape[0]):
        nf[i, i] = 0
    numedges = np.count_nonzero(nf)
    numvertices = nf.shape[0] - 2
    numslacks = numedges
    slack_counter = 0
    edge_counter = 0

    primal_constraints = np.zeros(
        (numedges + numvertices + 2, numedges + numslacks + 1))

    obj = np.zeros(numedges + numslacks)

    for i in range(numvertices + 2):
        for j in range(numvertices + 2):
            if nf[i, j] != 0:
                if i == 0:
                    obj[edge_counter] = -1
                    primal_constraints[edge_counter, edge_counter] = 1
                    primal_constraints[edge_counter,
                                       numedges + slack_counter] = 1
                    primal_constraints[edge_counter, -1] = nf[i, j]
                    primal_constraints[numedges + j, edge_counter] = 1
                    edge_counter += 1
                    slack_counter += 1
                elif j == numvertices + 1:
                    primal_constraints[edge_counter, edge_counter] = 1
                    primal_constraints[edge_counter,
                                       numedges + slack_counter] = 1
                    primal_constraints[edge_counter, -1] = nf[i, j]
                    primal_constraints[numedges + i, edge_counter] = -1
                    edge_counter += 1
                    slack_counter += 1
                else:
                    primal_constraints[edge_counter, edge_counter] = 1
                    primal_constraints[edge_counter,
                                       numedges + slack_counter] = 1
                    primal_constraints[edge_counter, -1] = nf[i, j]
                    primal_constraints[numedges + i, edge_counter] = -1
                    primal_constraints[numedges + j, edge_counter] = 1
                    edge_counter += 1
                    slack_counter += 1
    
    primal_constraints = np.vstack((primal_constraints[:numedges], primal_constraints[numedges+1:numedges+numvertices+1]))
    
    sign_constraints = np.hstack(
        (np.eye(2 * numedges), np.zeros(2 * numedges).reshape(1, 2 * numedges).T))

    LPMatrix = np.vstack((primal_constraints, sign_constraints))

    return LPMatrix, obj


In [8]:
class LPSolution(object):

    def __init__(self, num_vars=0, var_vals=list(), obj=0):

        self.num_vars = num_vars
        self.obj = obj
        self.var_vals = var_vals

    def __str__(self):

        sol = ""
        sol += "\tSolution to the LP is as follows:\n\n"
        sol += "optim\t:=\t" + str(self.obj) + "\n\n"
        for i in range(self.num_vars):
            if i in self.var_vals:
                sol += "x_" + str(i + 1) + "*\t:=\t" + \
                    str(self.var_vals[i]) + "\n"
            else:
                sol += "x_" + str(i + 1) + "*\t:=\t" + str(0.0) + "\n"
        return sol

In [9]:
class PrimalDual(object):
    
    def __init__(self, num_eq_constraints, num_vars, c, constraints=None, tol=10**-20, A=None, b=None):
        self.num_eq_constraints = num_eq_constraints
        self.num_vars = num_vars
        self.A = A
        self.b = b
        self.tol = tol
        self.constraints = constraints
        if constraints is not None:
            self.A = constraints[:self.num_eq_constraints, :-1]
            self.b = constraints[:self.num_eq_constraints, -1]
        self.c = c
        if A is not None:
            self.num_vars = A.shape[1]
        self.x = None
        self.s = None
        self.lamda = None
        self.obj = 0
        
    def fetch_constraints(self, filename):
        constraints = np.loadtxt(filename)
        self.A = constraints[:self.num_eq_constraints, :-1]
        self.b = constraints[:self.num_eq_constraints, -1]
    
    def run_PDBarrier(self):
        
        if self.A is None or self.b is None:
            raise RuntimeError("Please fetch the constraints first!")
        
        A = self.A.copy()
        b = self.b.copy()
        c = self.c.copy()
        
        
        np.random.seed(2010)
        x = np.random.random(A.shape[1])
        lamda = np.random.random(A.shape[0])
        s = np.random.random(A.shape[1])
        
        n = A.shape[1]
        mu = np.dot(x, s) / n
        e = np.ones(x.shape[0])
        eta = 0.9999
        
        while mu > self.tol:
            X = np.diag(x)
            S = np.diag(s)
            I = np.eye(x.shape[0])
            sigma = 1 - (1 / np.sqrt(n))

            bmatrix = np.block([
                [np.zeros((A.shape[1], A.shape[1])), A.T, I],
                [A, np.zeros((A.shape[0], A.shape[0])), np.zeros((A.shape[0], x.shape[0]))],
                [S, np.zeros((S.shape[0], A.shape[0])), X]
            ])
            rhs = -1 * np.hstack((
                np.dot(A.T, lamda) + s - c,
                np.dot(A, x) - b,
                np.dot(np.dot(X, S), e) - (sigma * mu * e)
            ))
            delta = np.linalg.solve(bmatrix, rhs)
            delx = delta[:x.shape[0]]
            dell = delta[x.shape[0]:-s.shape[0]]
            dels = delta[-s.shape[0]:]
            alphax = 1.0

            if delx[delx < 0].shape[0] > 0:
                alphax = np.minimum(1.0, eta * np.min(-1 * x[delx < 0] / delx[delx < 0]))
            
            alphas = 1.0
            if dels[dels < 0].shape[0] > 0:
                alphas = np.minimum(1.0, eta * np.min(-1 * s[dels < 0] / dels[dels < 0]))
            print(dell)
            x = x + (alphax * delx)
            s = s + (alphas * dels)
            lamda = lamda + (alphas * dell)
            mu = np.dot(s, x) / n
        
        self.x = x.copy()
        self.s = s.copy()
        self.lamda = lamda.copy()
        
#         self.x[self.x <= self.tol] = 0
#         self.s[self.s <= self.tol] = 0
#         self.lamda[self.lamda <= self.tol] = 0
        
        self.obj = np.dot(c, x)
        
        var_vals = {i: self.x[i] for i in range(self.num_vars)}
        return LPSolution(self.num_vars, var_vals, self.obj)
        

In [10]:
constraints, c = nf2PrimalLP("../assn3/nf1.dat")
pd = PrimalDual(14, 10, c, constraints, 10**-35)
sol = pd.run_PDBarrier()
print(sol)

[ 1.71461442  7.47003455  0.17977379  8.33031203 -0.08686258  5.57944018
 -0.29258438  8.08719654  1.0505778   4.19728127  5.07758021  3.31811223
  0.5953298   0.11422373]
[-0.8322004  -0.09206605 -0.32594535 -0.80343313  0.05666014  0.04494075
 -0.743643   -0.20688184 -0.95897283 -0.73077231 -2.40727637 -1.26199022
 -1.25926066 -1.46891336]
[-0.56624173 -0.42039423 -0.17425375 -1.27279114 -0.51860075 -0.29989799
 -0.46943955 -0.8136746  -1.14218142 -0.87176968 -0.8563387  -0.62465615
 -0.37659103 -0.81055831]
[-0.42583375 -0.32714998 -0.10561473 -0.98934908 -0.23642964 -0.37877313
 -0.2718523  -0.37080005 -0.7016495  -0.9314849  -0.59907861 -0.4401962
 -0.11946126 -0.37382187]
[-0.00377592 -0.00529068  0.01489227 -0.12338811  0.02007492 -0.00279412
 -0.01846435 -0.06817908 -0.17605732 -0.22065368 -0.12017145 -0.14336374
 -0.0460494  -0.19361306]
[ 0.02175929  0.01805191  0.01156921 -0.02516478  0.03530578 -0.00661706
  0.0070915   0.01793394 -0.0045348   0.01727755 -0.01670184 -0.0100

[ 4.27617676e-21  1.12319283e-20  1.91415524e-21  3.94210353e-20
  7.54273496e-21 -8.68316510e-05  1.42457426e-21  1.35525272e-20
  8.68316510e-05  8.68316510e-05 -3.29006366e-21 -1.02455539e-20
  1.28777288e-20  8.68316510e-05]
[ 3.32906887e-21  8.75144633e-21  1.49031570e-21  2.79546230e-20
  5.87308857e-21 -6.76128345e-05  1.10919038e-21  1.35525272e-20
  6.76128345e-05  6.76128345e-05 -2.56124757e-21 -7.98346555e-21
  1.30271212e-20  6.76128345e-05]
[ 2.63063316e-21  6.84102010e-21  1.17772155e-21  2.27884238e-20
  4.64151527e-21 -5.34358757e-05  8.76569066e-22  6.77626358e-21
  5.34358757e-05  5.34358757e-05 -2.02382695e-21 -6.23411529e-21
  6.36104665e-21  5.34358757e-05]
[ 2.02784187e-21  5.30374898e-21  9.07901391e-22  1.73028749e-20
  3.57832525e-21 -4.11966317e-05  6.75764273e-22  6.77626358e-21
  4.11966317e-05  4.11966317e-05 -1.56003429e-21 -4.83588294e-21
  6.45616471e-21  4.11966317e-05]
[ 1.58330090e-21  4.12947010e-21  7.08898786e-22  1.15143100e-20
  2.79410537e-21 -3

In [37]:
np.dot(pd.b, pd.lamda)

-26.0

In [8]:
constraints, c = nf2PrimalLP("../assn3/nf2.dat")
pd = PrimalDual(39, 58, c, constraints, 10**-30)

In [9]:
sol = pd.run_PDBarrier()

[ 5.66886602e+00  4.46384453e-01 -2.00549045e+00  6.61871661e+00
 -8.29536571e+00 -9.28652358e+00  1.05528648e+00 -4.81509503e+00
 -3.61857569e+00  1.28100024e+01  2.96719262e-01  7.00666224e+00
 -3.04649987e-01 -6.63990094e-01  8.70574910e+00  2.74281275e+01
  2.75945201e+01  4.60524116e+01 -5.78766545e-01  8.67280650e+00
  3.71145783e-03  1.06446705e+01 -4.09259531e-01 -5.62307087e-01
  4.41672460e+00  2.07100682e+00  2.99903754e+00 -2.27062923e+00
  1.46725000e+01  1.73097179e+01  6.32176887e+00  5.76820557e+00
  1.00508804e+01  1.04889845e+01  3.94500520e+01  9.53332315e+00
  1.05024342e+01  1.41515839e+00  9.65762244e+00]
[-0.2402797   0.73597993 -1.16700551  0.48235473 -1.47309178  1.51691428
  0.39356436  1.06462773 -1.66451009  4.72727759 -0.76597933  2.78297837
 -0.46149358 -1.11942994  2.19876783  0.90310555 -0.63380517 -2.27826949
 -0.37367316  4.53887054 -0.61635756  2.07299819 -0.43993485 -1.52696379
  0.83648415  0.99204486  0.08283637 -1.37990148  5.22827748 -3.45375366


[-1.58438472e-07  1.92976346e-08 -2.53537407e-07  1.95617102e-08
  2.61056828e-07 -1.28103114e-07 -2.42441235e-07  1.62044632e-07
  1.83906714e-07  5.05694007e-08  4.17003270e-08  1.19800542e-08
  2.72815959e-08  1.08937460e-07  3.68527185e-08  1.18256229e-08
  3.98134451e-08  7.22310496e-09  4.50404865e-08  3.09531888e-08
  1.38534577e-07  2.00808920e-07  1.34969807e-07  2.14712336e-07
  2.71268223e-07  1.15242133e-07  1.57541336e-07  8.96915122e-08
  5.95001105e-08  1.70718793e-07 -2.41218733e-09  2.67045761e-07
  1.31147772e-07  1.63332809e-07 -5.15493705e-08  1.42765982e-07
  2.32695303e-07  1.40951510e-07  4.88884238e-08]
[-1.37634513e-07  1.67637322e-08 -2.20246355e-07  1.69931327e-08
  2.26778414e-07 -1.11282389e-07 -2.10607184e-07  1.40767141e-07
  1.59758590e-07  4.39293185e-08  3.62248052e-08  1.04069961e-08
  2.36993466e-08  9.46332726e-08  3.20137224e-08  1.02728425e-08
  3.45856829e-08  6.27466489e-09  3.91263803e-08  2.68888362e-08
  1.20344111e-07  1.74441442e-07  1.1724

[-1.33605939e-12  1.62716864e-13 -2.13792985e-12  1.64943515e-13
  2.20122127e-12 -1.08019855e-12 -2.04432273e-12  1.36635378e-12
  1.55069383e-12  4.26399284e-13  3.51615217e-13  1.01015280e-13
  2.30037159e-13  9.18555834e-13  3.10740661e-13  9.97131092e-14
  3.35705105e-13  6.09048977e-14  3.79779291e-13  2.60995893e-13
  1.16811841e-12  1.69321357e-12  1.13805966e-12  1.81044633e-12
  2.28732375e-12  9.71717635e-13  1.32838338e-12  7.56274585e-13
  5.01702965e-13  1.43949546e-12 -2.04506303e-14  2.25172063e-12
  1.10583413e-12  1.37721709e-12 -4.34661598e-13  1.20379832e-12
  1.96207869e-12  1.18849875e-12  4.12225729e-13]
[-1.16052975e-12  1.41351093e-13 -1.85710946e-12  1.43285371e-13
  1.91218676e-12 -9.38056806e-13 -1.77557978e-12  1.18694274e-12
  1.34707776e-12  3.70410315e-13  3.05445877e-13  8.77513240e-14
  1.99831800e-13  7.97943544e-13  2.69938414e-13  8.66201366e-14
  2.91624865e-13  5.29076928e-14  3.29911827e-13  2.26725453e-13
  1.01473695e-12  1.47088373e-12  9.8862

[-8.50063051e-18  1.03536632e-18 -1.36029268e-17  1.04953448e-18
  1.40063562e-17 -2.52716676e-17 -2.40467521e-17  8.69409995e-18
  9.86705448e-18  2.71317579e-18  2.23732527e-18  6.42759551e-19
  1.46372490e-18  5.84476462e-18  1.97724075e-18  6.34473846e-19
  2.13608933e-18  3.87537456e-19  2.41653308e-18  1.66071512e-18
  7.43272964e-18  1.07739066e-17  7.24146599e-18  1.15198579e-17
  1.45542257e-17  6.18303280e-18  8.45249453e-18  4.81217012e-18
  3.19233261e-18  9.15949998e-18 -1.29420790e-19  1.43276833e-17
  7.03641517e-18  8.76322311e-18 -2.76574884e-18  7.65976063e-18
  1.24846935e-17  7.56240956e-18  2.62298955e-18]
[-7.38444304e-18  8.99416059e-19 -1.18167750e-17  9.11723854e-19
  1.21672315e-17 -2.19533351e-17 -2.08892589e-17  7.55250870e-18
  8.57144675e-18  2.35691835e-18  1.94355007e-18  5.58361087e-19
  1.27152841e-18  5.07730943e-18  1.71761632e-18  5.51163349e-19
  1.85560706e-18  3.36651295e-19  2.09922674e-18  1.44265254e-18
  6.45676443e-18  9.35922334e-18  6.2906

[-1.66793746e-22  2.03152726e-23 -2.66907627e-22  2.05932710e-23
  2.74823451e-22 -4.95863935e-22 -4.71829455e-22  1.70589875e-22
  1.93604812e-22  5.32361395e-23  4.38993157e-23  1.26118025e-23
  2.87202412e-23  1.14682103e-22  3.87961093e-23  1.24492259e-23
  4.19129311e-23  7.60400347e-24  4.74156127e-23  3.25854529e-23
  1.45840102e-22  2.11398464e-22  1.42087253e-22  2.26035027e-22
  2.85573384e-22  1.21319377e-22  1.65849253e-22  9.44212170e-23
  6.26378375e-23  1.79721646e-22 -2.53940908e-24  2.81128318e-22
  1.38063881e-22  1.71946164e-22 -5.42676932e-23  1.50294753e-22
  2.44966393e-22  1.48384595e-22  5.14665649e-23]
[-1.44892654e-22  1.76477466e-23 -2.31860939e-22  1.78892420e-23
  2.38737364e-22 -4.30753811e-22 -4.09875212e-22  1.48190327e-22
  1.68183255e-22  4.62458920e-23  3.81350531e-23  1.09557917e-23
  2.49490888e-23  9.96236051e-23  3.37019306e-23  1.08145624e-23
  3.64094936e-23  6.60554890e-24  4.11896378e-23  2.83067733e-23
  1.26690358e-22  1.83640486e-22  1.2343

[-1.40640325e-27  1.71298183e-28 -2.25056254e-27  1.73642262e-28
  2.31730869e-27 -4.18111160e-27 -3.97845642e-27  1.43841218e-27
  1.63247391e-27  4.48886615e-28  3.70158606e-28  1.06342597e-28
  2.42168797e-28  9.66998385e-28  3.27128419e-28  1.04971752e-28
  3.53409430e-28  6.41168839e-29  3.99807989e-28  2.74760225e-28
  1.22972233e-27  1.78250981e-27  1.19807834e-27  1.90592517e-27
  2.40795202e-27  1.02296382e-27  1.39843930e-27  7.96158791e-28
  5.28161641e-28  1.51541119e-27 -2.14122728e-29  2.37047127e-27
  1.16415331e-27  1.44984840e-27 -4.57584665e-28  1.26728391e-27
  2.06555427e-27  1.25117747e-27  4.33965579e-28]
[-1.22172608e-27  1.48804731e-28 -1.95503739e-27  1.50841005e-28
  2.01301900e-27 -3.63209769e-27 -3.45604756e-27  1.24953186e-27
  1.41811102e-27  3.89942560e-28  3.21552458e-28  9.23785720e-29
  2.10369206e-28  8.40020204e-28  2.84172637e-28  9.11877355e-29
  3.07002645e-28  5.56975882e-29  3.47308531e-28  2.38680998e-28
  1.06824543e-27  1.54844545e-27  1.0407

In [10]:
print(sol)

	Solution to the LP is as follows:

optim	:=	-29.0

x_1*	:=	11.0
x_2*	:=	8.0
x_3*	:=	10.0
x_4*	:=	11.09449633871313
x_5*	:=	3.482551770684198
x_6*	:=	3.0
x_7*	:=	5.0
x_8*	:=	5.166381308638481
x_9*	:=	2.265478443006163
x_10*	:=	8.328752432830271
x_11*	:=	0.7606121844749162
x_12*	:=	5.724299446081947
x_13*	:=	1.048547568055392
x_14*	:=	1.7599904027736808
x_15*	:=	9.334505935939449
x_16*	:=	0.5770481093973272
x_17*	:=	0.6070874872000932
x_18*	:=	2.2984161740867783
x_19*	:=	1.0008421263890366
x_20*	:=	4.6358775000183075
x_21*	:=	3.0249121697150576
x_22*	:=	2.3273039665766713
x_23*	:=	2.999157873610963
x_24*	:=	4.370863400577794
x_25*	:=	3.5020302440147955
x_26*	:=	5.827830216547376
x_27*	:=	8.14255321941038
x_28*	:=	0.4939093909288522
x_29*	:=	12.729696395911374
x_30*	:=	9.860761315262648e-31
x_31*	:=	6.999999999999999
x_32*	:=	9.675872040601473e-31
x_33*	:=	6.905503661286868
x_34*	:=	0.5174482293158019
x_35*	:=	7.888609052210118e-31
x_36*	:=	1.0908467205009304e-30
x_37*	:=	0.8336186913615

In [11]:
np.savetxt("A1.txt", pd.A)
np.savetxt("b1.txt", pd.b)
np.savetxt("c1.txt", pd.c)

In [25]:
constraints, c = nf2DualLP("../assn3/nf1.dat")
pd = PrimalDual(10, 24, c, constraints, 10**-35)
sol = pd.run_PDBarrier()
print(sol)

	Solution to the LP is as follows:

optim	:=	26.0

x_1*	:=	2.8735742344691386e-36
x_2*	:=	5.989383377505233e-17
x_3*	:=	1.7241445406814834e-36
x_4*	:=	1.0
x_5*	:=	5.953510223772712e-18
x_6*	:=	6.437432320626496e-20
x_7*	:=	9.578580781563795e-37
x_8*	:=	1.7465248547516345e-35
x_9*	:=	1.0
x_10*	:=	1.0
x_11*	:=	1.0
x_12*	:=	1.0
x_13*	:=	1.7023305412637085e-35
x_14*	:=	1.0
x_15*	:=	6.6313251564672425e-37
x_16*	:=	6.6313251564672425e-37
x_17*	:=	1.7241445406814827e-36
x_18*	:=	7.183935586172846e-37
x_19*	:=	2.1551806758518538e-36
x_20*	:=	6.15765907386244e-37
x_21*	:=	1.0
x_22*	:=	4.4194313487925974e-37
x_23*	:=	1.231531814772488e-36
x_24*	:=	1.231531814772488e-36



In [26]:
constraints, c = nf2DualLP("../assn3/nf2.dat")
pd = PrimalDual(29, 68, c, constraints, 10**-30)
sol = pd.run_PDBarrier()
print(sol)

	Solution to the LP is as follows:

optim	:=	29.0

x_1*	:=	1.0
x_2*	:=	1.330185270178119e-31
x_3*	:=	1.0
x_4*	:=	1.34240331292824e-31
x_5*	:=	2.7447794762824267e-30
x_6*	:=	1.0
x_7*	:=	1.0
x_8*	:=	1.162847857919234e-30
x_9*	:=	1.318713032783521e-30
x_10*	:=	3.7621242634543086e-31
x_11*	:=	2.850840374296305e-31
x_12*	:=	8.455212988199388e-32
x_13*	:=	1.8734546745408097e-31
x_14*	:=	7.559511696031214e-31
x_15*	:=	2.7918840931687135e-31
x_16*	:=	8.00093975876861e-32
x_17*	:=	2.570251283211861e-31
x_18*	:=	1.441653550129601e-31
x_19*	:=	3.0794181731496263e-31
x_20*	:=	2.198529380824206e-31
x_21*	:=	1.3331671378051388e-30
x_22*	:=	2.7125430587870283e-30
x_23*	:=	9.537522504540348e-31
x_24*	:=	1.9033731500853237e-30
x_25*	:=	3.258565188304688e-30
x_26*	:=	1.1766797414474958e-30
x_27*	:=	2.445659566931115e-30
x_28*	:=	5.8490198893131126e-31
x_29*	:=	1.3759314233336792e-30
x_30*	:=	2.5705571366951317e-30
x_31*	:=	1.0
x_32*	:=	3.2748629938418195e-30
x_33*	:=	2.291103753713132e-30
x_34*	:=	2.520

In [38]:
A = np.array([[1,2,2,1,0,0]\
            ,[2,1,2,0,1,0]\
            ,[2,2,1,0,0,1]])

c = np.array([-10,-12,-12,0,0,0])

b = np.array([20,20,20])
pdtest = PrimalDual(3, 6, np.array([-10, -12, -12, 0, 0, 0.0]), tol=10**-40, A=A, b=b)

In [39]:
sol = pdtest.run_PDBarrier()

(15, 15)
[-0.66137049 -3.84862542 -3.24885453]
(15, 15)
[-3.45594541 -2.27028821 -1.46566511]
(15, 15)
[-3.86347569 -1.14056763 -0.93339637]
(15, 15)
[-2.37841863 -1.30892184 -1.13101319]
(15, 15)
[0.00037045 0.00038685 0.00045875]
(15, 15)
[0.00020491 0.00019978 0.00020214]
(15, 15)
[0.00011954 0.00011979 0.00011979]
(15, 15)
[7.07751895e-05 7.08641658e-05 7.08641653e-05]
(15, 15)
[4.18944082e-05 4.19255653e-05 4.19255653e-05]
(15, 15)
[2.47956640e-05 2.48065744e-05 2.48065744e-05]
(15, 15)
[1.46744791e-05 1.46782996e-05 1.46782996e-05]
(15, 15)
[8.68420926e-06 8.68554710e-06 8.68554709e-06]
(15, 15)
[5.13909218e-06 5.13956065e-06 5.13956065e-06]
(15, 15)
[3.04113539e-06 3.04129944e-06 3.04129944e-06]
(15, 15)
[1.79962116e-06 1.79967860e-06 1.79967860e-06]
(15, 15)
[1.06493734e-06 1.06495745e-06 1.06495745e-06]
(15, 15)
[6.30181444e-07 6.30188489e-07 6.30188486e-07]
(15, 15)
[3.72911980e-07 3.72914448e-07 3.72914450e-07]
(15, 15)
[2.20671667e-07 2.20672527e-07 2.20672527e-07]
(15, 15)

In [24]:
np.dot(pdtest.b, pdtest.lamda)

-136.0

In [25]:
print(sol)

	Solution to the LP is as follows:

optim	:=	-136.0

x_1*	:=	4.000000000000002
x_2*	:=	3.9999999999999982
x_3*	:=	4.0
x_4*	:=	2.438018507346169e-41
x_5*	:=	5.48554164152888e-41
x_6*	:=	5.48554164152888e-41



In [28]:
print(pdtest.b)

[20 20 20]


In [3]:
constraints, c = nf2PrimalLP("../assn3/nf2.dat")

In [4]:
constraints

array([[ 1.,  0.,  0., ...,  0.,  0., 11.],
       [ 0.,  1.,  0., ...,  0.,  0., 15.],
       [ 0.,  0.,  1., ...,  0.,  0., 10.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  1.,  0.]])